# Imports

In [1]:
from datasets import load_dataset
from collections import Counter
import torch
import torch.nn as nn
from utils.datautils import *
from utils.MLutils import *
from utils.resources import *
from transformers import BertTokenizerFast
from sklearn.model_selection import train_test_split
from transformers import BertModel

# linea que arregla algunos errores de loadeo de datasets
# pip install --upgrade datasets

/home/fabro/Documents/uba/aprendizaje_automatico/Puntualizacion-Capitalizacion/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Procesamiento

In [2]:
linux = True
device = None

if linux:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
else:
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

print("usando:", device)

usando: cuda


# Ejericio b)

## Busqueda de fuentes

### Fuente 1: Conjunto de preguntas en espa;ol

In [3]:
questions, question_for_mixture = get_questions()

Se descargaron 5000 preguntas en Español.


### Fuente 2: Dataset provisto para Notebook 10

In [4]:
oraciones_rnn = get_notebook_dataset()

Se descargaron 997 oraciones en Español (del dataset del notebook 10).


### Fuente 3: Dataset sintetico generado con Gemini

In [5]:
oraciones_sinteticas = get_gemini_dataset()

Hay 1413 oraciones sintéticas.


### Fuente 4: Articulos de Wikipedia

In [6]:
frases_wikipedia = get_wikipedia_dataset()

['Argentina, oficialmente República Argentina,[a]\u200b es un país soberano de América del Sur, ubicado en el extremo sur y sudeste de ese subcontinente.', 'Adopta la forma de gobierno republicana, democrática, representativa y federal.', 'Poseen Carta Magna, bandera y fuerzas de seguridad propias, el dominio de los recursos naturales circunscriptos en su territorio y delegan los poderes exclusivos al Gobierno Federal.', 'Hasta mediados del siglo XX, fue una de las economías más prósperas del mundo.', 'No obstante, es la segunda economía más importante de Sudamérica —detrás de Brasil— y la 24.º más grande del mundo por PIB nominal.']


### Fuente 5: Subtitulos de peliculas

In [9]:
esperando_la_carroza, frases_relatos_salvajes = get_pelis_dataset()

✅ Se extrajeron 947 frases completas y se guardaron en 'dialogos_esperando_la_carroza.json'
✅ Frases extraídas y guardadas. Total: 947
['¡Ojalá que sea ella! ¡Ojalá que sea ella! Para que la conciencia le remuerda como se merece.', 'Ay, mirá, justo hoy estaba pensando en vos. Viste, creer o reventar...', 'A mi casa, por supuesto...', 'Ella es de una naturaleza muy peculiar, muy nerviosa. Pero vos, ¿estás segura de lo que estás diciendo? Porque yo no puedo creerlo, te juro, no puedo creerlo.', '¿Qué hacés, no me ves?', '¿vos qué tomás?', 'Pedí lo que quieras, tesoro, mirá que yo invito.', 'Debe tener 77, 78, me parece.', '¿Te das cuenta con lo que hay que lidiar, no?', 'Vas a ir a comprar: matambre, mortadela y salchichón o te mato.']
✅ Se extrajeron 1000 frases de Relatos Salvajes.


### Fuente 6: Mixture of preguntas y afirmaciones

In [10]:
import unicodedata


cant_oraciones = len(oraciones_sinteticas)
question_for_mixture = [re.sub(r'[\\\(\)!¡“]', '', unicodedata.normalize("NFC", q).strip()) for q in question_for_mixture]
oraciones_sinteticas = [re.sub(r'[\\\(\)!¡“]', '', unicodedata.normalize("NFC", a).strip()) for a in oraciones_sinteticas]

tanda_1 = question_for_mixture[:cant_oraciones]
question_affirmation = [f"{q} {a}" for q, a in zip(tanda_1, oraciones_sinteticas)]

tanda_2 = question_for_mixture[cant_oraciones:2*cant_oraciones]
affirmation_question = [f"{a} {q}" for q, a in zip(tanda_2, oraciones_sinteticas)]

tanda_3 = question_for_mixture[2*cant_oraciones:3*cant_oraciones]
tanda_3_shuffled = random.sample(tanda_3, len(tanda_3))
question_question = [f"{q} {p}" for q, p in zip(tanda_3, tanda_3_shuffled)]

mixtures = question_affirmation + affirmation_question + question_question

random.sample(mixtures, 5)

['¿Dónde tuvo lugar el acto de Giuliani? La cumbre del G7 se celebró en la ciudad de Cornualles.',
 '¿Sabes si el museo cierra los lunes por la tarde? ¿Quién escribió sobre cortar la cabeza de un roedor con los incisivos?',
 '¿Cómo ha sido premiado el film de Dumont? El Land Cruiser de Toyota es un todoterreno.',
 'Casio también fabrica teclados musicales. ¿Cuál era la estrategia de Bush?',
 '¿Cuántos discos vendieron en Estados Unidos? ¿Cuántas personas iban a bordo de la nave cuando desapareció?']

## Juntamos las fuentes

In [11]:
oraciones_raw = questions + oraciones_rnn + oraciones_sinteticas + frases_wikipedia + esperando_la_carroza  + frases_relatos_salvajes + mixtures

print('Cantidad total de oraciones:',len(oraciones_raw))
print('Cantidad de oraciones de preguntas:',len(questions))
print('Cantidad de oraciones en espa;ol de hugging face:',len(oraciones_rnn))
print('Cantidad de oraciones sintéticas:',len(oraciones_sinteticas))
print('Cantidad de oraciones de Wikipedia:',len(frases_wikipedia))
print('Cantidad de oraciones de Esperando la carroza:',len(esperando_la_carroza))
print('Cantidad de oraciones de Relatos Salvajes:',len(frases_relatos_salvajes))
print('Cantidad de oraciones de mixture:',len(mixtures))

print("Algunas oraciones aleatorias:")
random.sample(oraciones_raw, 5)

Cantidad total de oraciones: 20244
Cantidad de oraciones de preguntas: 5000
Cantidad de oraciones en espa;ol de hugging face: 997
Cantidad de oraciones sintéticas: 1413
Cantidad de oraciones de Wikipedia: 6648
Cantidad de oraciones de Esperando la carroza: 947
Cantidad de oraciones de Relatos Salvajes: 1000
Cantidad de oraciones de mixture: 4239
Algunas oraciones aleatorias:


['Participó un histórico 86 % del electorado, y la opción Rechazo triunfó con un 61,87 %.',
 '¿Cuál era el linaje al que pertenecía Claudio? Mi amiga de Chile, Camila, me recomendó visitar el desierto.',
 'Durkheim buscó distinguir a la sociología de la filosofía por un lado y de la psicología por el otro.',
 'Tu hace la cartas',
 'Izuzquiza Otero, Ignacio; Corellano Aznar, Luis; Frechilla García, Ana Rosa; Peña Calvo, José Vicente; Villamayor Lloro, Santiago (2008).']

Separamos en conjuntos de `train` y `test` con el tokenizer de `BERT`

In [12]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")

train_sents, test_sents = train_test_split(oraciones_raw, test_size=0.05, random_state=42)

dataloader_train = get_dataloader(oraciones_raw=oraciones_raw, max_length=64, batch_size=64, device=device, tokenizer=tokenizer)
dataloader_test = get_dataloader(oraciones_raw=test_sents, max_length=64, batch_size=64, device=device, tokenizer=tokenizer)

print(len(train_sents))
print(len(test_sents))

19231
1013


## Importamos el modelo

In [ ]:
from train.PunctuationCapitalizationRNN import PunctuationCapitalizationRNN

model_name = "bert-base-multilingual-cased"
bert_model = BertModel.from_pretrained(model_name)

bert_embeddings = bert_model.embeddings.word_embeddings
for param in bert_embeddings.parameters():
    param.requires_grad = False

# Congelar la mayoría de los parámetros de BERT salvo los últimos N layers y el pooler
N = 2
for layer in bert_model.encoder.layer[-N:]:
    for param in layer.parameters():
        param.requires_grad = True

for param in bert_model.pooler.parameters():
    param.requires_grad = True


# Crear el modelo
model = PunctuationCapitalizationRNN(
    bert_model = bert_model,
    hidden_dim=256,
    num_punct_classes=len(PUNCT_TAGS),
    num_cap_classes=len(CAP_TAGS)
).to(device)


# le quiero pasar el weight param al criterion para mejorar el desbalanceo de clases en base a un counter de tokens
# Contar ocurrencias de cada etiqueta de puntuación y capitalización
punct_counter = Counter()
cap_counter = Counter()
for input_ids, attention_mask, punct_labels, cap_labels in dataloader_train:
    punct_labels_np = punct_labels.cpu().numpy()
    cap_labels_np = cap_labels.cpu().numpy()

    # Contar etiquetas válidas (ignorando -100)
    valid_punct = punct_labels_np[punct_labels_np != -100]
    valid_cap = cap_labels_np[cap_labels_np != -100]

    punct_counter.update(valid_punct)
    cap_counter.update(valid_cap)

# como las clases estan desbalanceadas, calculamos los pesos inversos
total_punct = sum(punct_counter.values())
total_cap = sum(cap_counter.values())
beta = 0.7

total_punct = sum(punct_counter.values())
total_cap   = sum(cap_counter.values())

punct_weights = {
    tag: (total_punct / count)**beta
    for tag, count in punct_counter.items()
}
cap_weights = {
    tag: (total_cap / count)**beta
    for tag, count in cap_counter.items()
}

punct_weights_tensor = torch.tensor(
    [punct_weights.get(i, 1.0) for i in range(len(PUNCT_TAGS))],
    dtype=torch.float32
).to(device).clamp(min=1.0, max=5.0)

cap_weights_tensor = torch.tensor(
    [cap_weights.get(i, 1.0) for i in range(len(CAP_TAGS))],
    dtype=torch.float32
).to(device).clamp(min=1.0, max=5.0)


criterion_punct = nn.CrossEntropyLoss(ignore_index=-100, weight=punct_weights_tensor)
criterion_cap   = nn.CrossEntropyLoss(ignore_index=-100, weight=cap_weights_tensor)
trainable_params = [
    p for p in bert_model.parameters() if p.requires_grad
] + list(model.projection.parameters()) \
  + list(model.rnn.parameters()) \
  + list(model.punct_classifier.parameters()) \
  + list(model.cap_classifier.parameters())

optimizer = torch.optim.AdamW(trainable_params, lr=2e-5)
# Entrenamiento 
train(model, dataloader_train=dataloader_train, dataloader_test=dataloader_test,optimizer=optimizer, criterion_punct=criterion_punct, criterion_cap = criterion_cap, device=device, epochs=20)

## Evaluacion

In [ ]:
evaluate(model, dataloader_test, device)

In [ ]:
entrada = "hola como estas"
print(f"{entrada} => {predict_and_reconstruct(model, entrada, tokenizer, device, verbose=False)}")

## Prueba de control de Overfitting

explicacion de del corro

In [ ]:
frases = ["Buenas tardes, quiero un APPLE por favor. Muchisimas HH"]

train_loader = get_dataloader(frases, max_length=25, batch_size=1, device=device)

model = PunctuationCapitalizationRNN(
    bert_model=bert_model,
    hidden_dim=64,
    num_punct_classes=5,
    num_cap_classes=4
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)  # Alto LR
criterion_punct = nn.CrossEntropyLoss(ignore_index=-100)
criterion_cap = nn.CrossEntropyLoss(ignore_index=-100)

train(model, train_loader, train_loader,optimizer, criterion_punct, criterion_cap, device, epochs=200)

entrada = "buenas tardes quiero un apple por favor muchisimas hh"
print("Predicción:", predict_and_reconstruct(model, entrada, tokenizer, device))


## Export modelo

In [ ]:
torch.save(model, "modelo_fine_tuned.pt")

In [ ]:
torch.save(model.state_dict(), "modelo_fine_tuned_state_dict.pt")

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

## Evaluacion modelo entrenado

In [ ]:
# Cargar la clase del modelo
from train.PunctuationCapitalizationRNNBidirectional import PunctuationCapitalizationRNNBidirectional

model_name = "bert-base-multilingual-cased"
bert_model = BertModel.from_pretrained(model_name)

tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")

# Crear la instancia
model = PunctuationCapitalizationRNNBidirectional(
    bert_model = bert_model,
    hidden_dim=256,
    num_punct_classes=len(PUNCT_TAGS),
    num_cap_classes=len(CAP_TAGS)
).to(device)

# Cargar los pesos en CPU
state_dict = torch.load("models/modelo_fine_tuned_state_dict_bidirec.pt", map_location=device)
model.load_state_dict(state_dict)
model.eval()

PunctuationCapitalizationRNNBidireccional(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [7]:
entrada = "hola como estas"
print(f"{entrada} => {predict_and_reconstruct(model, entrada, tokenizer, device, verbose=False)}")

hola como estas => Hola, como estas.
